In [1]:
import numpy as np
import pandas as pd
import sklearn as skl
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import csv
from datetime import datetime
import random

import pylab

from scipy import stats

from tensorflow.contrib.layers import dropout

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [135]:
def standardize_numpy(array):
    
    #if np.linalg.norm(list) == 0:
    #    return list
    #else:
    normalized_array = stats.zscore(array)
    return normalized_array

with open('C:/Users/Robin Köster/Desktop/Studienarbeit ML/08_06_EINS/GANs/Energy_Data/2015_2017_hourly_DEATLU_load_PV_wind_forecast_noNA.csv', newline = '') as csvfile:
    datareader = csv.reader(csvfile, delimiter = ';')
    
   # C:\Users\Robin Köster\Desktop\Studienarbeit ML\Energy_data
    
    
    data_array = np.array(list(datareader))    

date_np_array = np.array([np.array(data_array[i]) for i in range(1,len(data_array))])

prices = np.array(list(map(lambda it: float(date_np_array[it,4]) , range(1,len(date_np_array))) ))
    
prices_mean = np.mean(prices)
prices_std = np.std(prices)
    
prices_standardized = standardize_numpy(prices)


load_forecast = (np.array((\
                                        list(map(\
                                                 lambda it: float(date_np_array[it,1]) ,\
                                                 range(1,len(date_np_array))) ))) )

load_max = np.max(load_forecast)

load_forecast = load_forecast / load_max

wind_forecast = (np.array((\
                                        list(map(\
                                                 lambda it: float(date_np_array[it,3]) , \
                                                 range(1,len(date_np_array))) ))) )

wind_max = np.max(wind_forecast)

wind_forecast = wind_forecast / wind_max

pv_forecast = (np.array(( \
                                      list(map( \
                                               lambda it: float(date_np_array[it,2]) , \
                                               range(1,len(date_np_array))) ))) )

pv_max =     np.max(pv_forecast)
    
pv_forecast = pv_forecast / pv_max


date_array = np.array(list(map(lambda it: datetime.strptime(date_np_array[it,0],'%d.%m.%Y %H:%M') , \
                               range(1,len(date_np_array))) ))

day_of_week_array = (np.array((\
                        list(map(lambda date: datetime.weekday(date), date_array)))) )

hour_array = np.array(list(map(lambda date: datetime.time(date).hour, date_array)))


In [150]:
tf.reset_default_graph()

In [151]:
# network architecture of the generative net:

# one LSTM network, which gets the prices of the previous day as input
# one MLP, which gets just the day of the week as input

# hyperparameters
lrate = 1e-3
mb_size = 50                  # size of mini batches
conditions_length = 24
time_series_pred_length = 24  # number of hours in the predicted time series as output
time_series_pred_dim = 1
n_LSTM = 50                   # number of neurons in the LSTM layer (number of neurons in each "step" in the unrolled RNN)
n_DNN = 128 * 3                     # number of neurons in the hidden layer of the DNN

z_length = 96                 # number of randomly generated inputs

no_its = 5000000

d_h1_LSTM = 24


def sample_z(m, n):
    random.seed( datetime.now() )
    return np.random.uniform(-1., 1., size=[m, n])



def next_batch(size):
    
    ind_vec =  np.where(hour_array == 0)[0]   # indices of data points with hour 0
    ind_vec = list(ind_vec[:-2])                     # skip the last two elements
    
    random.seed( datetime.now() )
    
    ind_vec_shuffled = random.sample(ind_vec,len(ind_vec))

    mb_ind = ind_vec_shuffled[0:(size)]
    
    pv_forecast_batch = \
        np.array((list(map(lambda mb_first_ind: \
                           pv_forecast[(mb_first_ind):\
                                   (mb_first_ind+time_series_pred_length)], mb_ind))))
    
    wind_forecast_batch = \
        np.array((list(map(lambda mb_first_ind: \
                           wind_forecast[(mb_first_ind):\
                                   (mb_first_ind+time_series_pred_length)], mb_ind))))
    load_forecast_batch = \
        np.array((list(map(lambda mb_first_ind: \
                           load_forecast[(mb_first_ind):\
                                   (mb_first_ind+time_series_pred_length)], mb_ind))))
    
    forecasts = np.concatenate((pv_forecast_batch, wind_forecast_batch, load_forecast_batch), axis = 1)
    
    prices_batch = \
        np.array((list(map(lambda mb_first_ind: \
                           prices_standardized[(mb_first_ind):\
                                   (mb_first_ind+time_series_pred_length)], mb_ind))))
        
    data_batch = np.concatenate((forecasts,prices_batch),axis = 1)
    
    return data_batch


# inputs for the generator
z_random_nodes = tf.placeholder(tf.float32, shape=[None, z_length])

# input nodes for the forecasts
future_input_nodes = tf.placeholder(tf.float32, shape=[None, conditions_length * 4])

def xavier_init(size):
    in_dim = size[0]
    random.seed( datetime.now() )
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

with tf.variable_scope("generator_vars"):

##########    DNN    #############################

    DNN_h1 = tf.contrib.layers.fully_connected(\
                                                   z_random_nodes, n_DNN)

########## forecast vector #######################

#     forecast_pred = tf.contrib.layers.fully_connected(DNN_h1, 72)
    
########## LSTM ##################################

    lstm_input =  tf.contrib.layers.fully_connected(\
                                                   z_random_nodes, d_h1_LSTM)

    lstm_input = lstm_input[:,:,np.newaxis]
    
# create the LSTM cell
    lstm_cell = tf.contrib.rnn.OutputProjectionWrapper(\
                                tf.contrib.rnn.BasicLSTMCell \
                                (num_units = n_LSTM), output_size = 1)

# include the cell in the graph
    lstm_output, lstm_states = tf.nn.dynamic_rnn(\
                            lstm_cell, lstm_input, dtype = tf.float32)
    
    lstm_output = tf.squeeze(lstm_output, axis =[2])
    
########## merge ##################################

    merged_states = tf.concat((DNN_h1, lstm_output), axis = 1)
    
    forecast_pred = tf.contrib.layers.fully_connected(merged_states, 24*3)
    
    prices_pred = tf.contrib.layers.fully_connected(merged_states, 24, activation_fn=None)
    
    merged_pred = tf.concat((forecast_pred, prices_pred), axis = 1)

In [152]:
# network architecture of the discriminative net:  

dim_discr_h1 = (128 * 1)

real_data = future_input_nodes


D_W1 = tf.Variable(xavier_init([24*4, dim_discr_h1]), name='D_W1')
D_b1 = tf.Variable(tf.zeros(shape=[dim_discr_h1]), name='D_b1')

D_W2 = tf.Variable(xavier_init([dim_discr_h1, 1]), name='D_W2')
D_b2 = tf.Variable(tf.zeros(shape=[1]), name='D_b2')

theta_D = [D_W1, D_W2, D_b1, D_b2]

def D(X):
    D_h1 = tf.nn.relu(tf.matmul(X, D_W1) + D_b1)
    D_logit = tf.matmul(D_h1, D_W2) + D_b2
    D_prob = tf.nn.sigmoid(D_logit)
    return D_prob


In [153]:
# defining the loss functions

optimizer = tf.train.AdamOptimizer()


######## DISCRIMINATOR ##########################################


D_real = D(real_data)
D_fake = D(merged_pred)

    
D_loss = - tf.reduce_mean(tf.log(D_real) + tf.log(1. - D_fake) , axis = 0)

D_training_op = optimizer.minimize(D_loss, var_list=theta_D)

######## GENERATOR ###############################################

D_fake_log = tf.log(D_fake)

G_loss = - tf.reduce_mean(tf.log(D_fake), axis = 0)

G_train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                                     "generator_vars")

G_training_op = optimizer.minimize(G_loss, var_list=G_train_vars)

In [ ]:
# training procedure

saver = tf.train.Saver()

if not os.path.exists('out/'):
    os.makedirs('out/')
    
if not os.path.exists('cpkt_files/'):
    os.makedirs('cpkt_files/')

sess = tf.Session()

sess.run(tf.global_variables_initializer())

In [ ]:
i=0

for it in range(no_its):
        
    # get the batches for random sampling and sampling from the real data
    z_batch = sample_z(mb_size, z_length)
        
    data_batch = next_batch(mb_size)
        
    sess.run(D_training_op, feed_dict = {z_random_nodes:z_batch,\
                                            future_input_nodes:data_batch})
    sess.run(G_training_op, feed_dict = {z_random_nodes:z_batch})
        
    if it % 500 == 0:
        z_batch = sample_z(1, z_length)
        data_batch = next_batch(1)
        
        D_real_print, D_fake_print, G_loss_print, D_loss_print, scenario_pred = \
                        sess.run([D_real, D_fake, G_loss, D_loss, merged_pred], feed_dict = {z_random_nodes:z_batch,\
                                             future_input_nodes:data_batch})
        
        scenario_pred = np.squeeze(scenario_pred) 
        
        pv_pred = scenario_pred[0:23] * pv_max
        wind_pred = scenario_pred[24:47] * wind_max
        load_pred = scenario_pred[48:71] * load_max
        prices_pred = scenario_pred[72:95] * prices_std + prices_mean
        
        print(D_real_print)
        print(D_fake_print)
        print(data_batch)
        
        fig = plt.figure(figsize=(15,8))
        fig = plt.plot(pv_pred, label='PV forecast')
        fig = plt.plot(wind_pred, label='Wind forecast')
        fig = plt.plot(load_pred, label='Load forecast')
        plt.grid()
        pylab.legend(loc='upper left')

        plt.savefig('out/{}.png'
                    .format(str(i).zfill(4)), bbox_inches='tight')
        fig = plt.gcf()
        plt.close(fig)
        
        fig = plt.figure(figsize=(15,8))
        fig = plt.plot(prices_pred, label='Price')
        plt.grid()
        pylab.legend(loc='upper left')
        plt.savefig('out/{}_price.png'
                    .format(str(i).zfill(4)), bbox_inches='tight')
        i += 1
        fig = plt.gcf()
        plt.close(fig)
        
    if it % 10000 == 0:
        save_path = saver.save(sess, "cpkt_files/model.cpkt")
        
save_path = saver.save(sess, "cpkt_files/model_final.cpkt")

[[0.57049143]]
[[0.4980417]]
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  4.72332231e-04  2.24176144e-02  1.25713040e-01
   2.97060640e-01  4.78363550e-01  6.19954220e-01  7.06427352e-01
   7.38509610e-01  7.18707990e-01  6.56069469e-01  5.59423028e-01
   4.34182320e-01  2.89176325e-01  1.53362642e-01  5.51538713e-02
   8.93797914e-03  3.63332486e-05  0.00000000e+00  0.00000000e+00
   1.90768545e-01  1.94977039e-01  1.98022008e-01  2.00745151e-01
   2.05547785e-01  2.09112627e-01  2.08567998e-01  1.96313855e-01
   1.75221874e-01  1.63735162e-01  1.68166458e-01  1.89233683e-01
   2.21614329e-01  2.53697904e-01  2.76572306e-01  2.93950909e-01
   3.04769214e-01  3.04670190e-01  3.00783523e-01  3.05610912e-01
   3.20192105e-01  3.40739457e-01  3.53290671e-01  3.50542772e-01
   5.84563189e-01  5.61377049e-01  5.48981741e-01  5.50737913e-01
   5.64399299e-01  6.08370698e-01  6.92929511e-01  7.71023743e-01
   8.15415224e-01  8.32011342e-01  8.52014785e-

[[0.50696963]]
[[0.33763376]]
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  8.02964793e-03
   7.67358209e-02  1.91912219e-01  3.01892962e-01  3.83715438e-01
   4.28587000e-01  4.37670312e-01  4.11437707e-01  3.44221197e-01
   2.34204120e-01  1.08273081e-01  1.67132943e-02  1.45332994e-04
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   2.45677010e-01  2.55133743e-01  2.67907760e-01  2.80137147e-01
   2.91500080e-01  2.98085135e-01  3.01278639e-01  3.03135328e-01
   3.01848024e-01  3.07145775e-01  3.24969984e-01  3.32817587e-01
   3.25762171e-01  3.16552996e-01  3.15859832e-01  3.24598646e-01
   3.34377205e-01  3.38239117e-01  3.43041751e-01  3.59751946e-01
   3.80546857e-01  3.89582740e-01  3.86513015e-01  3.72204137e-01
   5.29917647e-01  5.09893784e-01  5.01678866e-01  5.06095551e-01
   5.22960054e-01  5.73725681e-01  6.86482434e-01  7.70889551e-01
   8.15187680e-01  8.25462162e-01  8.34234271e

In [149]:
a = [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  1.08999746e-04
   5.99498601e-03  4.21465683e-02  9.80271046e-02  1.47440323e-01
   1.63499619e-01  1.51582313e-01  1.10889075e-01  4.76692221e-02
   3.88765760e-03  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   2.89915706e-01  3.03506666e-01  3.17048113e-01  3.29698350e-01
   3.40887992e-01  3.57004048e-01  3.68589783e-01  3.79556623e-01
   3.93692211e-01  3.96588644e-01  3.94707200e-01  3.79556623e-01
   3.59430120e-01  3.33585018e-01  3.00634987e-01  2.76646573e-01
   2.61396972e-01  2.56396292e-01  2.57089455e-01  2.58500538e-01
   2.59639307e-01  2.57832131e-01  2.56445803e-01  2.46865291e-01
   6.08691593e-01  5.85380013e-01  5.72322494e-01  5.66429690e-01
   5.65464087e-01  5.64416802e-01  5.65140275e-01  5.91415761e-01
   6.17621233e-01  6.48415507e-01  6.73894589e-01  6.98618108e-01
   6.97938394e-01  6.78655507e-01  6.62575739e-01  6.57908171e-01
   6.73926678e-01  7.21705062e-01  7.34943450e-01  7.26810214e-01
   7.01797889e-01  6.81517309e-01  6.76663040e-01  6.39272910e-01
  -1.09681894e+00 -8.59246694e-01 -1.00849957e+00 -1.29057382e+00
  -1.42339519e+00 -1.42065661e+00 -1.48912123e+00 -1.34739946e+00
  -1.10640398e+00 -8.63354571e-01 -8.17483273e-01 -6.85346550e-01
  -6.68915041e-01 -7.38064310e-01 -7.90097424e-01 -6.94931598e-01
  -5.25823978e-01 -2.51280838e-01  2.32764050e-01  9.58348032e-02
  -1.07505128e-01 -3.79309683e-01 -3.19745461e-01 -5.80595676e-01]

SyntaxError: invalid syntax (<ipython-input-149-8b50356c5bbb>, line 1)